 # Naive Bayes

 ## Importações

In [4]:
import pandas as pd
from collections import Counter
import numpy as np

 ## Leitura de Dados

In [3]:
dados = pd.read_csv("diabetes_train.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'diabetes_train.csv'

In [3]:
teste = pd.read_csv("diabetes_test.csv")

In [4]:
labels_classes_secundarias = list(dados.columns[0:len(dados.columns) - 1])
label_classe_principal = dados.columns[len(dados.columns) - 1]

In [5]:
x_treinamento = pd.DataFrame(dados.loc[:][[i for i in labels_classes_secundarias]])
y_treinamento = pd.DataFrame(dados.loc[:, label_classe_principal])

In [6]:
x_teste = pd.DataFrame(teste.loc[:][[i for i in labels_classes_secundarias]])
y_teste = pd.DataFrame(teste.loc[:, label_classe_principal])

## Algoritmo (Sem Correção Laplaciana)

In [7]:
class NaiveBayes: 
    def __init__(self):
        self.tabela_bayesiana = {}
        self.categorias_classe_principal = None
        self.labels_classes_secundarias = None
        self.qtd_classe_principal = None

    def inicializa_tabela_bayesiana(self, x):
        for i in self.categorias_classe_principal:
            self.tabela_bayesiana[i] = {}
            for j in self.labels_classes_secundarias:
                self.tabela_bayesiana[i][j] = {}

                for k in list(Counter(x.loc[:][j]).keys()):
                    self.tabela_bayesiana[i][j][k] = 0

    def calcula_tabela_bayesiana(self, x, y, i):
        for j, k in zip(x, y):
            self.tabela_bayesiana[k][i][j] += 1
    
    def fit(self, x, y):
        if not isinstance(x_treinamento, pd.DataFrame) or not isinstance(y_treinamento, pd.DataFrame):
            print("Atenção: os dados de treinamento precisam ser dataframes!\n")
            return
        
        self.label_classe_principal = y.columns[0]
        self.labels_classes_secundarias = list(x.columns[0:len(x.columns)])
        self.qtd_classe_principal = dict(Counter(y.loc[:, label_classe_principal]))
        self.categorias_classe_principal = list(Counter(y.loc[:, label_classe_principal]).keys())
        self.inicializa_tabela_bayesiana(x)
        
        for i in self.labels_classes_secundarias:
            self.calcula_tabela_bayesiana(list(x.loc[:][i].values), list(y.loc[:, label_classe_principal].values), i)

        for i in self.categorias_classe_principal:
            for j in self.tabela_bayesiana[i]:
                for k in self.tabela_bayesiana[i][j]:
                    self.tabela_bayesiana[i][j][k] /= self.qtd_classe_principal[i]

    def calcula_probabilidade(self, x):
        p = []
        for i in self.categorias_classe_principal:
            produtorio = 1
            for j in self.labels_classes_secundarias:
                produtorio *= self.tabela_bayesiana[i][j][x[j]]
            
            p.append(produtorio * self.qtd_classe_principal[i]) 

        for i, k in enumerate(p):
            if k == max(p):
                return self.categorias_classe_principal[i]

    def predict(self, x):
        resultados = []
        for i in range(len(x)):
            r = self.calcula_probabilidade(x.iloc[i])
            resultados.append(r)
        return resultados

In [8]:
naive_bayes = NaiveBayes()
naive_bayes.fit(x_treinamento, y_treinamento)
y_resultado = naive_bayes.predict(x_teste)

In [9]:
acuracia = 0
for i, k in zip(y_teste.loc[:, label_classe_principal], y_resultado):
    if i == k:
        acuracia += 1
acuracia = (acuracia/len(y_teste.loc[:, label_classe_principal])) * 100

In [10]:
print(f"Acurácia: {acuracia:.2f}%")

Acurácia: 88.46%


In [11]:
matriz_confusao = np.zeros((2, 2))

for i, k in zip(y_teste.loc[:, label_classe_principal], y_resultado):
    matriz_confusao[i][k] += 1

In [12]:
matriz_confusao

array([[ 74.,   7.],
       [ 17., 110.]])